In [ ]:
import os
import pandas as pd
import numpy as np
import multiprocessing
from collections import Counter
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix

## Read the created dataset and attach column values

In [ ]:
dataset = pd.read_csv('output-data/cpa-datasets/dataset.csv')
dataset

In [ ]:
col_table = (dataset['column_name']+'|'+dataset['file_name']).tolist()

### Add value columns

In [ ]:
#Existing English Tables
existing = open("output-data/english_table_names.txt", 'r')
existing_english_tables = [line.replace('\n', '') for line in existing.readlines()]

In [ ]:
#Returns values of cleaned textual columns
def get_values(col_table_name):
    column_name, file_name = col_table_name.split('|')
    
    if file_name in existing_english_tables:
        file = 'output-data/expanded-tables/' + file_name
    else:
        file = 'output-data/new-english-tables/' + file_name
    
    #Open table
    df = pd.read_json(file, compression='gzip', lines=True)
           
    return df[column_name].tolist()

In [ ]:
pool = multiprocessing.Pool(processes=25)
values = pool.map(get_values, col_table)
pool.close()
pool.join()

In [ ]:
dataset['values'] = values
dataset = dataset.loc[~dataset['relation_label'].isin(['hasMenu', 'acceptsReservations'])]

## Create training, validation and test split

In [ ]:
grouped_by_table = dataset.groupby(['file_name'])['relation_label'].apply(','.join).reset_index()
grouped_by_table

In [ ]:
X = grouped_by_table[["file_name"]].values

### One hot encoding of CPA labels

In [ ]:
all_labels = dataset['relation_label'].unique()
y = np.zeros(shape=(len(grouped_by_table['file_name'].tolist()), len(all_labels))) #encoded labels

In [ ]:
for index, row in grouped_by_table.iterrows():
    table_labels = row['relation_label'].split(',')
    count = 0
    
    for label in all_labels:
        if label in table_labels:
            y[index][count] = 1
        else:
            y[index][count] = 0
        count += 1

In [ ]:
X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size = 0.2)
print('Training set length: '+str(len(X_train)) +', Testing set length: '+ str(len(X_test)))

In [ ]:
#Relation Labels in each set
pd.DataFrame({
    'train': Counter(str(combination) for row in get_combination_wise_output_matrix(y_train, order=1) for combination in row),
    'test' : Counter(str(combination) for row in get_combination_wise_output_matrix(y_test, order=1) for combination in row)
}).T.fillna(0.0)

### Split testing set into validation and testing

In [ ]:
X_val, y_val, X_test, y_test = iterative_train_test_split(X_test, y_test, test_size = 0.5)
print('Validation set length: '+str(len(X_val)) +', Testing set length: '+ str(len(X_test)))

In [ ]:
pd.DataFrame({
    'val': Counter(str(combination) for row in get_combination_wise_output_matrix(y_val, order=1) for combination in row),
    'test' : Counter(str(combination) for row in get_combination_wise_output_matrix(y_test, order=1) for combination in row)
}).T.fillna(0.0)

## Statistics for each set

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
grouped_by_table_dict = grouped_by_table.to_dict('records')

In [ ]:
#Dictionary with table names as key and relation labels as values
file_to_label = {}
for row in grouped_by_table_dict:
    file_to_label[row['file_name']] = row['relation_label']

### Training set

In [ ]:
#Count how many columns per label
label_and_number_train = {}
for row in X_train:
    
    for label in file_to_label[row[0]].split(','):
        if label in label_and_number_train:
            label_and_number_train[label] += 1
        else:
            label_and_number_train[label] = 1
print('Number of unique relation labels in training set: '+ str(len(label_and_number_train)))

In [ ]:
label_and_number_train.values()

In [ ]:
print('Minimum column count per label is: '+str(min(label_and_number_train.values())) )
print('Maximum column count per label is: '+str(max(label_and_number_train.values())) )
print('Total column count is: '+str(sum(label_and_number_train.values())) )

In [ ]:
plt.figure(figsize=(15,5))
plt.hist(label_and_number_train.values(), bins=10)
plt.ylabel('Label Count')
plt.xlabel('Number of columns')

### Validation set

In [ ]:
#Count how many columns per label
label_and_number_val = {}
for row in X_val:
    
    for label in file_to_label[row[0]].split(','):
        if label in label_and_number_val:
            label_and_number_val[label] += 1
        else:
            label_and_number_val[label] = 1
print('Number of unique relation labels in validation set: '+ str(len(label_and_number_val)))

In [ ]:
label_and_number_val.values()

In [ ]:
print('Minimum column count per label is: '+str(min(label_and_number_val.values())) )
print('Maximum column count per label is: '+str(max(label_and_number_val.values())) )
print('Total column count is: '+str(sum(label_and_number_val.values())) )

In [ ]:
plt.figure(figsize=(15,5))
plt.hist(label_and_number_val.values(), bins=10)
plt.ylabel('Label Count')
plt.xlabel('Number of columns')

### Testing set

In [ ]:
#Count how many columns per label
label_and_number_test = {}
for row in X_test:
    
    for label in file_to_label[row[0]].split(','):
        if label in label_and_number_test:
            label_and_number_test[label] += 1
        else:
            label_and_number_test[label] = 1
print('Number of unique relation labels in testing set: '+ str(len(label_and_number_test)))

In [ ]:
label_and_number_test.values()

In [ ]:
print('Minimum column count per label is: '+str(min(label_and_number_test.values())) )
print('Maximum column count per label is: '+str(max(label_and_number_test.values())) )
print('Total column count is: '+str(sum(label_and_number_test.values())) )

In [ ]:
plt.figure(figsize=(15,5))
plt.hist(label_and_number_test.values(), bins=10)
plt.ylabel('Label Count')
plt.xlabel('Number of columns')

## Prepare csv file for each set

In [ ]:
training_tables = [ table[0] for table in X_train ]
validation_tables = [ table[0] for table in X_val ]
testing_tables = [ table[0] for table in X_test ]

In [ ]:
training_set = dataset.loc[dataset['file_name'].isin(training_tables)]
validation_set = dataset.loc[dataset['file_name'].isin(validation_tables)]
testing_set = dataset.loc[dataset['file_name'].isin(testing_tables)]

training_set = training_set.loc[training_set['relation_label'].isin(testing_set['relation_label'].unique())]
validation_set = validation_set.loc[validation_set['relation_label'].isin(testing_set['relation_label'].unique())]


In [ ]:
rel_lbls = {}
for index, row in testing_set.iterrows():
    rel_lbls[row['column_name']] = row['relation_label']

selection = {}
for index, row in testing_set.iterrows():
    if row['file_name'] not in selection:
        selection[row['file_name']] = {}
        
    selection[row['file_name']][row['column_name']] = row['selection_type']

In [ ]:
# Some manual transfer to Random
selection_count = {
    'Random': {},
    'Value Heterogeneity': {},
    'Corner Cases': {},
    'Missing values': {}
}
for r in testing_set['relation_label'].unique():
    print(r)
    
    selection_count['Random'][r] = []
    selection_count['Value Heterogeneity'][r] = []
    selection_count['Corner Cases'][r] = []
    selection_count['Missing values'][r] = []
for file in selection:
    for col in selection[file]:
        sel_type = selection[file][col]
        selection_count[sel_type][rel_lbls[col]].append(col + '|' + file)
transfer = []
#alternateName
for c in selection_count['Missing values']['alternateName']:
    transfer.append(c)
#additonalName
transfer.append(selection_count['Missing values']['additionalName'][0])
#homeLocation
for c in selection_count['Corner Cases']['homeLocation'][:6]:
    transfer.append(c)
#episodeNumber
for c in selection_count['Corner Cases']['episodeNumber']:
    transfer.append(c)
#alumniOf
for c in selection_count['Missing values']['alumniOf'][:2]:
    transfer.append(c)
#checkInTime
for c in selection_count['Missing values']['checkInTime'][:3]:
    transfer.append(c)
#checkOutTime
for c in selection_count['Missing values']['checkoutTime'][:3]:
    transfer.append(c)
#honorificSuffix
for c in selection_count['Missing values']['honorificSuffix']:
    transfer.append(c)
#homeTeam
for c in selection_count['Corner Cases']['homeTeam']:
    transfer.append(c)
#awayTeam
for c in selection_count['Corner Cases']['awayTeam']:
    transfer.append(c)
#award
for c in selection_count['Missing values']['award']:
    transfer.append(c)
#memberOf
for c in selection_count['Missing values']['memberOf'][:2]:
    transfer.append(c)
transfer.append(selection_count['Corner Cases']['memberOf'][0])
#deathDate
for c in selection_count['Value Heterogeneity']['deathDate'][:8]:
    transfer.append(c)

i = 0
for file in selection:
    for col in selection[file]:
        if col + '|'+ file in transfer:
            i += 1
            selection[file][col] = 'Random'

new_selection_types = []
for index, row in testing_set.iterrows():
    new_selection_types.append(selection[row['file_name']][row['column_name']])

testing_set['selection_type'] = new_selection_types

In [ ]:
training_set.to_csv('output-data/cpa-datasets/training_set.csv.gz', index=False, compression='gzip')
validation_set.to_csv('output-data/cpa-datasets/validation_set.csv.gz', index=False, compression='gzip')
testing_set.to_csv('output-data/cpa-datasets/testing_set.csv.gz', index=False, compression='gzip')

In [ ]:
with open('output-data/cpa-datasets/relation_vocab.txt', 'a') as file:
    i = 0
    for label in testing_set['relation_label'].unique():
        file.write(str(i)+ '\t' + label +'\n')
        i += 1

### Create small subset of training set

In [ ]:
grouped_by_table = training_set.groupby(['file_name'])['relation_label'].apply(','.join).reset_index()
grouped_by_table

In [ ]:
X = grouped_by_table[["file_name"]].values

### One hot encoding of CPA labels

In [ ]:
r = open("output-data/cpa-datasets/relation_vocab.txt", 'r')
all_labels = [line.replace('\n', '').split('\t')[1] for line in r.readlines()]
y = np.zeros(shape=(len(grouped_by_table['file_name'].tolist()), len(all_labels))) #encoded labels

In [ ]:
for index, row in grouped_by_table.iterrows():
    table_labels = row['relation_label'].split(',')
    count = 0
    
    for label in all_labels:
        if label in table_labels:
            y[index][count] = 1
        else:
            y[index][count] = 0
        count += 1

In [ ]:
X_rest, y_rest, X_test, y_test = iterative_train_test_split(X, y, test_size = 0.25)
print('Training set length: '+str(len(X_rest)) +', Testing set length: '+ str(len(X_test)))

In [ ]:
#Relation Labels in each set
pd.DataFrame({
    'train': Counter(str(combination) for row in get_combination_wise_output_matrix(y_rest, order=1) for combination in row),
    'test' : Counter(str(combination) for row in get_combination_wise_output_matrix(y_test, order=1) for combination in row)
}).T.fillna(0.0)

In [ ]:
training_tables = [ table[0] for table in X_test ]
training_set_small = training_set.loc[training_set['file_name'].isin(training_tables)]
training_set_small

In [ ]:
training_set_small.to_csv('output-data/cpa-datasets/training_set_small.csv.gz', index=False, compression='gzip')